In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install ../input/talib-package/talib_binary-0.4.19-cp37-cp37m-manylinux1_x86_64.whl
import talib as ta 

In [ ]:
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from lightgbm import early_stopping
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler
import missingno as msno

from pylab import rcParams

from tqdm import tqdm

# import ta
from talib import abstract


import time
import gc
import sys

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GroupKFold

import seaborn as sns
import plotly.graph_objects as go

from decimal import ROUND_HALF_UP, Decimal

# sys.path.insert(0, '../input/jpx-local-api')
# from local_api import local_api

In [ ]:
data = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
# display(data)
train = data.copy()

# using supplement data as test data
supp_data = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')

train_with_supp = pd.concat([train, supp_data]).reset_index(drop=True)
# train_with_supp = train.copy()
# train_with_supp


In [ ]:
# def time_elapsed(t0):
#     t1 = time.time()
#     print(time.time() - t0)
#     return t1

SecuritiesCode 에 따라 인덱싱하는 것이 시간 소요가 너무 크다.  
미리 Data를 종목마다 쪼개서 넣어 놓는게 좋겠다. - > 리스트 활용

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
def generate_adjusted_close(df):
    """
    Args:
        df (pd.DataFrame)  : stock_price for a single SecuritiesCode
    Returns:
        df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
    """
    # sort data to generate CumulativeAdjustmentFactor
    df = df.sort_values("Date", ascending=False)
    # generate CumulativeAdjustmentFactor
    df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
    # generate AdjustedClose
    df.loc[:, "AdjustedClose"] = (
        df["CumulativeAdjustmentFactor"] * df["Close"]
    ).map(lambda x: float(
        Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
    ))
    # reverse order
    df = df.sort_values("Date")
    # to fill AdjustedClose, replace 0 into np.nan
    df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
    # forward fill AdjustedClose
    df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()

    df = df.drop(columns=['CumulativeAdjustmentFactor'])
    
    return df

In [ ]:
def get_ta_features(df, inf=None, train=True):
    """
    Get technical features from TA-Lib
    ref : https://www.kaggle.com/code/daosword/jpx-pytorch-neural-network-with-ta-lib-features
    """
    if train:
        op = df['Open']
        hi = df['High']
        lo = df['Low']
        cl = df['AdjustedClose']  #df['Close']
        vo = df['Volume']

    #     # Overlap Studies
    #     df['BBANDS_upper'], df['BBANDS_middle'], df['BBANDS_lower'] = ta.BBANDS(cl, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    #     df['DEMA'] = ta.DEMA(cl, timeperiod=30)
        for period in [7,15,30,90]:
            df['EMA{}'.format(period)] = cl.ewm(span=period,  adjust=False).mean().iloc[-1]/cl
            df['return{}'.format(period)] = cl.pct_change(period)
            df['volatility{}'.format(period)] = np.log(cl).diff().rolling(period).std()
            
            


        # df['EMA7'] = ta.EMA(cl, 7)/cl
        # df['EMA15'] = ta.EMA(cl, 15)/cl
        # df['EMA30'] = ta.EMA(cl, 30)/cl
        # df['EMA90'] = ta.EMA(cl, 90)/cl

    #     df['HT_TRENDLINE'] = ta.HT_TRENDLINE(cl)
    #     df['KAMA'] = ta.KAMA(cl, timeperiod=30)
    #     df['MA'] = ta.MA(cl, timeperiod=30, matype=0)
    #     df['MIDPOINT'] = ta.MIDPOINT(cl, timeperiod=14)
    #     df['SAR'] = ta.SAR(hi, lo, acceleration=0, maximum=0)
    #     df['SAREXT'] = ta.SAREXT(hi, lo, startvalue=0, offsetonreverse=0, accelerationinitlong=0, accelerationlong=0, accelerationmaxlong=0, accelerationinitshort=0, accelerationshort=0, accelerationmaxshort=0)
    #     df['SMA'] = ta.SMA(cl, timeperiod=30)
    #     df['T3'] = ta.T3(df['Close'], timeperiod=5, vfactor=0)
    #     df['TEMA'] = ta.TEMA(df['Close'], timeperiod=30)
    #     df['TRIMA'] = ta.TRIMA(df['Close'], timeperiod=30)
    #     df['WMA'] = ta.WMA(df['Close'], timeperiod=30)

        # Momentum Indicators
    #     df['ADX'] = ta.ADX(hi, lo, cl, timeperiod=14)
    #     df['ADXR'] = ta.ADXR(hi, lo, cl, timeperiod=14)
    #     df['APO'] = ta.APO(cl, fastperiod=12, slowperiod=26, matype=0)
    #     df['AROON_down'], df['AROON_up'] = ta.AROON(hi, lo, timeperiod=14)
    #     df['AROONOSC'] = ta.AROONOSC(hi, lo, timeperiod=14)
    #     df['BOP'] = ta.BOP(op, hi, lo, cl)
    #     df['CCI'] = ta.CCI(hi, lo, cl, timeperiod=14)
    #     df['DX'] = ta.DX(hi, lo, cl, timeperiod=14)
    #     df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = ta.MACD(cl, fastperiod=12, slowperiod=26, signalperiod=9)
    #     df['MFI'] = ta.MFI(hi, lo, cl, vo, timeperiod=14)
    #     df['MINUS_DI'] = ta.MINUS_DI(hi, lo, cl, timeperiod=14)
    #     df['MINUS_DM'] = ta.MINUS_DM(hi, lo, timeperiod=14)
        df['MOM'] = ta.MOM(cl, timeperiod=10)
    #     df['PLUS_DI'] = ta.PLUS_DI(hi, lo, cl, timeperiod=14)
    #     df['PLUS_DM'] = ta.PLUS_DM(hi, lo, timeperiod=14)
        df['RSI'] = ta.RSI(cl, timeperiod=14)
        df['STOCH_slowk'], df['STOCH_slowd'] = ta.STOCH(hi, lo, cl, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        df['STOCHF_fastk'], df['STOCHF_fastd'] = ta.STOCHF(hi, lo, cl, fastk_period=5, fastd_period=3, fastd_matype=0)
        df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = ta.STOCHRSI(cl, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    #     df['TRIX'] = ta.TRIX(cl, timeperiod=30)
    #     df['ULTOSC'] = ta.ULTOSC(hi, lo, cl, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    #     df['WILLR'] = ta.WILLR(hi, lo, cl, timeperiod=14)

        # Volume Indicators
    #     df['AD'] = ta.AD(hi, lo, cl, vo)
    #     df['ADOSC'] = ta.ADOSC(hi, lo, cl, vo, fastperiod=3, slowperiod=10)
    #     df['OBV'] = ta.OBV(cl, vo)

        # Volatility Indicators
        df['ATR'] = ta.ATR(hi, lo, cl, timeperiod=14)
        df['NATR'] = ta.NATR(hi, lo, cl, timeperiod=14)
        df['TRANGE'] = ta.TRANGE(hi, lo, cl)

        # Cycle Indicators
    #     df['HT_DCPERIOD'] = ta.HT_DCPERIOD(cl)
    #     df['HT_DCPHASE'] = ta.HT_DCPHASE(cl)
    #     df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = ta.HT_PHASOR(cl)
    #     df['HT_SINE_sine'], df['HT_SINE_leadsine'] = ta.HT_SINE(cl)
    #     df['HT_TRENDMODE'] = ta.HT_TRENDMODE(cl)

        # Statistic Functions
    #     df['BETA'] = ta.BETA(hi, lo, timeperiod=5)
    #     df['CORREL'] = ta.CORREL(hi, lo, timeperiod=30)
    #     df['LINEARREG'] = ta.LINEARREG(cl, timeperiod=14) - cl
    #     df['LINEARREG_ANGLE'] = ta.LINEARREG_ANGLE(cl, timeperiod=14)
    #     df['LINEARREG_INTERCEPT'] = ta.LINEARREG_INTERCEPT(cl, timeperiod=14) - cl
    #     df['LINEARREG_SLOPE'] = ta.LINEARREG_SLOPE(cl, timeperiod=14)
        df['STDDEV'] = ta.STDDEV(cl, timeperiod=5, nbdev=1)   
    
    
    else: #inference 
        op = inf['Open']
        hi = inf['High']
        lo = inf['Low']
        cl = inf['Close']
        vo = inf['Volume']
        
            #     # Overlap Studies
    #     df['BBANDS_upper'], df['BBANDS_middle'], df['BBANDS_lower'] = ta.BBANDS(cl, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    #     df['DEMA'] = ta.DEMA(cl, timeperiod=30)

        # df['EMA7'] = cl.ewm(span=7, min_periods=3, adjust=False).mean().iloc[-1]/cl
        # df['EMA15'] = cl.ewm(span=15, min_periods=7, adjust=False).mean().iloc[-1]/cl
        # df['EMA30'] = cl.ewm(span=30, min_periods=15, adjust=False).mean().iloc[-1]/cl
        # df['EMA90'] = cl.ewm(span=90, min_periods=30, adjust=False).mean().iloc[-1]/cl

        df['EMA7'] = (ta.EMA(cl, 7)/cl).iloc[-1]
        df['EMA15'] = (ta.EMA(cl, 15)/cl).iloc[-1]
        df['EMA30'] = (ta.EMA(cl, 30)/cl).iloc[-1]
        df['EMA90'] = (ta.EMA(cl, 90)/cl).iloc[-1]

    #     df['HT_TRENDLINE'] = ta.HT_TRENDLINE(cl)
    #     df['KAMA'] = ta.KAMA(cl, timeperiod=30)
    #     df['MA'] = ta.MA(cl, timeperiod=30, matype=0)
    #     df['MIDPOINT'] = ta.MIDPOINT(cl, timeperiod=14)
    #     df['SAR'] = ta.SAR(hi, lo, acceleration=0, maximum=0)
    #     df['SAREXT'] = ta.SAREXT(hi, lo, startvalue=0, offsetonreverse=0, accelerationinitlong=0, accelerationlong=0, accelerationmaxlong=0, accelerationinitshort=0, accelerationshort=0, accelerationmaxshort=0)
    #     df['SMA'] = ta.SMA(cl, timeperiod=30)
    #     df['T3'] = ta.T3(df['Close'], timeperiod=5, vfactor=0)
    #     df['TEMA'] = ta.TEMA(df['Close'], timeperiod=30)
    #     df['TRIMA'] = ta.TRIMA(df['Close'], timeperiod=30)
    #     df['WMA'] = ta.WMA(df['Close'], timeperiod=30)

        # Momentum Indicators
    #     df['ADX'] = ta.ADX(hi, lo, cl, timeperiod=14)
    #     df['ADXR'] = ta.ADXR(hi, lo, cl, timeperiod=14)
    #     df['APO'] = ta.APO(cl, fastperiod=12, slowperiod=26, matype=0)
    #     df['AROON_down'], df['AROON_up'] = ta.AROON(hi, lo, timeperiod=14)
    #     df['AROONOSC'] = ta.AROONOSC(hi, lo, timeperiod=14)
    #     df['BOP'] = ta.BOP(op, hi, lo, cl)
    #     df['CCI'] = ta.CCI(hi, lo, cl, timeperiod=14)
    #     df['DX'] = ta.DX(hi, lo, cl, timeperiod=14)
    #     df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = ta.MACD(cl, fastperiod=12, slowperiod=26, signalperiod=9)
    #     df['MFI'] = ta.MFI(hi, lo, cl, vo, timeperiod=14)
    #     df['MINUS_DI'] = ta.MINUS_DI(hi, lo, cl, timeperiod=14)
    #     df['MINUS_DM'] = ta.MINUS_DM(hi, lo, timeperiod=14)
        df['MOM'] = ta.MOM(cl, timeperiod=10).iloc[-1]
    #     df['PLUS_DI'] = ta.PLUS_DI(hi, lo, cl, timeperiod=14)
    #     df['PLUS_DM'] = ta.PLUS_DM(hi, lo, timeperiod=14)
        df['RSI'] = ta.RSI(cl, timeperiod=14).iloc[-1]
        df['STOCH_slowk'], df['STOCH_slowd'] = (ta.STOCH(hi, lo, cl, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[0].iloc[-1],
                                                ta.STOCH(hi, lo, cl, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[1].iloc[-1])
        df['STOCHF_fastk'], df['STOCHF_fastd'] = (ta.STOCHF(hi, lo, cl, fastk_period=5, fastd_period=3, fastd_matype=0)[0].iloc[-1],
                                                  ta.STOCHF(hi, lo, cl, fastk_period=5, fastd_period=3, fastd_matype=0)[1].iloc[-1])
        df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = (ta.STOCHRSI(cl, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)[0].iloc[-1],
                                                      ta.STOCHRSI(cl, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)[1].iloc[-1])
    #     df['TRIX'] = ta.TRIX(cl, timeperiod=30)
    #     df['ULTOSC'] = ta.ULTOSC(hi, lo, cl, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    #     df['WILLR'] = ta.WILLR(hi, lo, cl, timeperiod=14)

        # Volume Indicators
    #     df['AD'] = ta.AD(hi, lo, cl, vo)
    #     df['ADOSC'] = ta.ADOSC(hi, lo, cl, vo, fastperiod=3, slowperiod=10)
    #     df['OBV'] = ta.OBV(cl, vo)

        # Volatility Indicators
        df['ATR'] = ta.ATR(hi, lo, cl, timeperiod=14).iloc[-1]
        df['NATR'] = ta.NATR(hi, lo, cl, timeperiod=14).iloc[-1]
        df['TRANGE'] = ta.TRANGE(hi, lo, cl).iloc[-1]

        # Cycle Indicators
    #     df['HT_DCPERIOD'] = ta.HT_DCPERIOD(cl)
    #     df['HT_DCPHASE'] = ta.HT_DCPHASE(cl)
    #     df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = ta.HT_PHASOR(cl)
    #     df['HT_SINE_sine'], df['HT_SINE_leadsine'] = ta.HT_SINE(cl)
    #     df['HT_TRENDMODE'] = ta.HT_TRENDMODE(cl)

        # Statistic Functions
    #     df['BETA'] = ta.BETA(hi, lo, timeperiod=5)
    #     df['CORREL'] = ta.CORREL(hi, lo, timeperiod=30)
    #     df['LINEARREG'] = ta.LINEARREG(cl, timeperiod=14) - cl
    #     df['LINEARREG_ANGLE'] = ta.LINEARREG_ANGLE(cl, timeperiod=14)
    #     df['LINEARREG_INTERCEPT'] = ta.LINEARREG_INTERCEPT(cl, timeperiod=14) - cl
    #     df['LINEARREG_SLOPE'] = ta.LINEARREG_SLOPE(cl, timeperiod=14)
        df['STDDEV'] = ta.STDDEV(cl, timeperiod=5, nbdev=1).iloc[-1]   
    
    return df

In [ ]:
def divideSecurities(df):
    sec_list = []
#     print('Divide securities individually..')
    for code in np.sort(df.SecuritiesCode.unique()):
        sec_list.append(df.loc[df.SecuritiesCode == code, :].reset_index(drop=True))
    return sec_list

sec_list = divideSecurities(train)

In [ ]:
def add_features_train(sec_list):
    df_list = []
    for df in tqdm(sec_list):
        
        # shadows
        df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
        df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']
        
        # adjusted close
        df = generate_adjusted_close(df)

        ## Rolling features ##
        
        # lagged features
        # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
        # lagged close, target (target 은 정확히 무엇? return인가)
        
        # lagged feature 계산하기 전 결측치 채워넣기
        df = df.fillna(method='ffill')
        # df = df.interpolate()
        
        
        
        # All indicators in ta
        df = get_ta_features(df.copy())
        # try:
        #     df = get_ta_features(df.copy())
        # except:
        #     print(f"error in SecuritiesCode: {df['SecuritiesCode'][0]}")
        #     display(df)
        #     continue
                
        # not add pattern recognition - feature importance 가 거의 0임. 
#         for indicator in ta.get_function_groups()['Pattern Recognition']:
#             df[str(indicator)] = getattr(ta,str(indicator))(df.Open, df.High, df.Low, df.Close)


        ####  For test!! df['STOCHF_fastd'] df['STOCHRSI_fastk'] df['ATR']
        ####  open high low close 등 제거
        df = df.drop(columns=['STOCHF_fastd', 'STOCHRSI_fastk', 'STOCH_slowk', 'NATR', 'ATR'])
    #     df = df.drop(columns=['upper_shadow', 'Low', 'TRANGE', 'High',
    #    'STOCHRSI_fastk', 'STOCHRSI_fastd', 'Open'])
        # fill ema features by backward -- 이렇게 채워진 것은 false data 이므로 일단 test 해보고 없애는 것을 검토하자.
        # df = df.fillna(method='ffill')
        # df = df.interpolate()
        
        # volatility
        
        df_list.append(df)
        
        del df
        
    gc.collect()
    df_feature_added = pd.concat(df_list).sort_values(['Date','SecuritiesCode'])
    
    return df_feature_added




In [ ]:
# def add_features_train(input_df, sec_list):
#     df_list = []
#     for df in sec_list:
        
#         # shadows
#         df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
#         df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']
        

        
#         # lagged features
#         # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
#         # lagged close, target (target 은 정확히 무엇? return인가)
        
#         # lagged feature 계산하기 전 결측치 채워넣기
#         df = df.fillna(method='ffill')
        

        
#         # TA-lib features - RSI, EMA 7-90
#         df['RSI'] = ta.RSI(df['Close'])
#         df['EMA7'] = ta.EMA(df['Close'], 7)
#         df['EMA15'] = ta.EMA(df['Close'], 15)
#         df['EMA30'] = ta.EMA(df['Close'], 30)
#         df['EMA90'] = ta.EMA(df['Close'], 90)
        

        
# #         for indicator in ta.get_function_groups()['Pattern Recognition']:
# #             df[str(indicator)] = getattr(ta,str(indicator))(df.Open, df.High, df.Low, df.Close)


#         # fill ema features by backward -- 이렇게 채워진 것은 false data 이므로 일단 test 해보고 없애는 것을 검토하자.
#         df = df.fillna(method='bfill')

    
#         # volatility
        
#         df_list.append(df)

        
#     df_feature_added = pd.concat(df_list)
    
#     return df_feature_added


In [ ]:
# add feature to data

df_added = add_features_train(sec_list)
# df_added = df_added[(df_added['Date'] >= '2021-06-03') & (df_added['Date'] <= '2021-12-03')]
# df_added = df_added[df_added['Date'] >= '2021-05-27']
# df_added.to_csv('train_with_supp_feature_added.csv', index=False)

# load data
# df_added = pd.read_csv('/kaggle/input/train-with-supp-feature-added-v1-all-cdl/train_with_supp_feature_added.csv')

# df_added

In [ ]:
def add_features_infer(input_df, close_df): #input df 는 price 데이터
    
    df_list = []
    
    print('Divide input securities...')
    sec_list = divideSecurities(input_df)
    print('Divide close_df securities...')
    close_list = divideSecurities(close_df) # for rolling features
    print('='*10 + 'feature adding' + '='*10)
    
    for i in range(len(sec_list)):
        
        
        close = close_list[i] #.loc[close_df.SecuritiesCode == code, :].fillna(method='ffill')
        df = sec_list[i]
#         # test data의 open, high, low, close 중 nan 있으면 이전 값에서 가져와 채움
        if df.loc[:, ['Open', 'High', 'Low', 'Close']].isna().any().any():
#             display(df)
#             display(close)
# #             print(close.iloc[-2]['Open', 'High', 'Low', 'Close'].values())
            df.loc[:, ['Open', 'High', 'Low', 'Close']] = close.loc[close['Date'] == close.iloc[-1]['Date'], ['Open', 'High', 'Low', 'Close']].values 
        
        

        
        # shadows
        df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
        df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']

        # lagged features
        # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
        # lagged close, target (target 은 정확히 무엇? return인가)
        
        ## Rolling features ##
        
        
        # TA-lib features
        df = get_ta_features(df, close, train=False)
        
        ####  open high low close 등 제거
        df = df.drop(columns=['STOCHF_fastd', 'STOCHRSI_fastk', 'ATR', 'Open', 'High', 'Low', 'Close'])
        
#         for indicator in ta.get_function_groups()['Pattern Recognition']:
#             df[str(indicator)] = getattr(ta,str(indicator))(df.Open, df.High, df.Low, df.Close)


        # volatility
        
        df_list.append(df)
        del df
    
    df_feature_added = pd.concat(df_list)
    
    return df_feature_added



In [ ]:
def preprocess_train(df):
    
    # remove columns - Date removed temporarily
    dfc = df.drop(columns=['RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'])
    
    
#     minmax = MinMaxScaler()
    stdsc = StandardScaler()
    ordinal = OrdinalEncoder()

    target = ['Target']
#     minmax_features = ['Date']
    ord_features = ['SecuritiesCode'] 
    scaled_features = [i for i in df.columns if i not in ['Date', 'RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
                                                         'Target', 'SecuritiesCode']]
    
#     date_scaled = minmax.fit_transform(dfc.loc[:,minmax_features])
    date_code_ord = ordinal.fit_transform(dfc.loc[:,ord_features])
    scaled = stdsc.fit_transform(dfc.loc[:,scaled_features])
    
#     display(pd.DataFrame(date_code_ord, columns=ord_features))
#     display(pd.DataFrame(scaled, columns=scaled_features))
    
    
    dfc_scaled = pd.concat([df['Date'].reset_index(drop=True),
                            pd.DataFrame(date_code_ord, columns=ord_features),
                            pd.DataFrame(scaled, columns=scaled_features)], axis=1)

    dfc_scaled = dfc_scaled.set_index(['Date']).drop(columns=['Close'])

    y = dfc.set_index(['Date']).loc[:, 'Target']
    
    return dfc_scaled, y, [ordinal, stdsc]
    

X_scaled, y, trained_scalers = preprocess_train(df_added)  # 2021-12-06부터 test 시작이므로 그 전까지만 이용한다.

# X_scaled

In [ ]:
X_scaled

In [ ]:
def preprocess_inference(df, trained_scalers: list):
    ordinal = trained_scalers[0]
    stdsc = trained_scalers[1]
    
      
    # remove columns - Date removed temporarily
    dfc = df.drop(columns=['RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'])
    
    
    target = ['Target']
    ord_features = ['SecuritiesCode'] 
    scaled_features = [i for i in df.columns if i not in ['Date', 'RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
                                                         'Target', 'SecuritiesCode']]
#     print('scaled_features:', scaled_features)
    date_code_ord = ordinal.transform(dfc.loc[:,ord_features])
    scaled = stdsc.transform(dfc.loc[:,scaled_features])
    


    dfc_scaled = pd.concat([df['Date'].reset_index(drop=True),
                            pd.DataFrame(date_code_ord, columns=ord_features),
                            pd.DataFrame(scaled, columns=scaled_features)], axis=1)

    dfc_scaled = dfc_scaled.set_index(['Date'])

    

#     y = dfc.loc[:, ['Target']]
    
    return dfc_scaled


# X_test_scaled = preprocess_train(df_added, trained_scalers)

# X_test_scaled

In [ ]:
params = {
 'n_estimators': 1000,
 'learning_rate': 0.0497652883951982,
 'num_leaves': 227,
 'max_depth': 12,
 'min_data_in_leaf': 500,
 'lambda_l1': 3.060184062092395e-08,
 'lambda_l2': 0.22368531324354038,
 'bagging_fraction': 0.4,
 'feature_fraction': 0.30000000000000004,
 'seed': 2022
}

130일 훈련 60일 테스트 로 6-fold 진행  
총 1202일 - 190일로 6개로 나누고, 5개로 앙상블, 1개로 예측


In [ ]:
def preprocess_train_lgb(df):
    
    # remove columns - Date removed temporarily
    dfc = df.drop(columns=['RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag']).set_index(['Date']).drop(columns=['Target'])

    y = df.set_index('Date').loc[:, ['Target']]
    
    return dfc, y
    

X_lgb, y = preprocess_train_lgb(df_added)  # 2021-12-06부터 test 시작이므로 그 전까지만 이용한다.


In [ ]:
X_lgb.columns

In [ ]:
score_board = dict()

In [ ]:
# base model - lgbm - only 130 days for training
# test - time-series cv 5-fold ensemble
class EnsembleModel:
    def __init__(self, models: list):
        self.models = models
    
    def predict(self, x):
        predicted = np.zeros((len(x), len(self.models)))
        for i, model in enumerate(self.models):
            predicted[:, i] = model.predict(x)
        return np.sum(predicted, axis=1) / len(self.models)
        
def group_timeseriesCV(grid, train_size=200, test_size=60):
    # 6-fold
    cv_size = train_size + test_size
    for i in range(4):
        cv_train = grid[cv_size*i:cv_size*(i+1)][:train_size]
        cv_test = grid[cv_size*i:cv_size*(i+1)][-test_size:]
        yield cv_train, cv_test


grid = train['Date'].unique()[90:] #90일의 false data 고려하지 않기
tscv = TimeSeriesSplit(n_splits=10, test_size=60)

k = 0
feat_importance = pd.DataFrame()
models = []
cv_score = []
# for train_idx, eval_idx in group_timeseriesCV(grid): #  #

for train_idx, eval_idx in tscv.split(grid):
    train_idx = grid[train_idx]
    eval_idx = grid[eval_idx]

    k += 1
    t0 = time.time()
    print(f'========Training fold {k}========')
   
    print('training size:', len(train_idx), '  test size:', len(eval_idx))
    
    print("Train Date range: {} to {}".format(train_idx.min(),train_idx.max()))
    print("Valid Date range: {} to {}".format(eval_idx.min(),eval_idx.max()))
    

    X, y = X_scaled.copy(), y.copy()
    # X, y = X_lgb.copy(), y.copy()

    X_train, X_test = X.loc[train_idx].reset_index(drop=True), X.loc[eval_idx].reset_index(drop=True)
    y_train, y_test = y.loc[train_idx].reset_index(drop=True), y.loc[eval_idx].reset_index(drop=True)

    
    lgb = LGBMRegressor(device='GPU')
    lgb.fit(
        X_train, 
        y_train,
        eval_set = [(X_test, y_test)],
        callbacks=[
                early_stopping(stopping_rounds=20)
            ]
        )
    
    models.append(lgb)

    feat_importance["Importance_Fold"+str(k)]=lgb.feature_importances_
    feat_importance.set_index(X_train.columns, inplace=True)

    y_pred = lgb.predict(X_test)
   
    
    # Sharpe Ratio 
    df_sharpe = pd.DataFrame([])
    df_sharpe['Date'] = X.loc[eval_idx].index
    df_sharpe['predict'] = y_pred
    df_sharpe['Target'] = y_test
    df_sharpe['Rank'] = (df_sharpe.groupby('Date')['predict'].rank(method='first', ascending=False)-1).astype(int)
#         display(X_test.groupby('Date')['Rank'].min())
#         display(X_test.groupby('Date')['Rank'].max())
#         display(X_test)
    eval_score = calc_spread_return_sharpe(df_sharpe)
    cv_score.append(eval_score)

    print(f'Fold {k} evaluated in {time.time() - t0 :.3f}s')
    print(f'Fold {k} sharpe ratio score: {eval_score :.6f}')
    
print(f'cv mean sharpe ratio score: {np.mean(cv_score) :.6f}')

# score_type = 'scaled_6fold_basicfeatonly'
# if score_type in score_board.keys():
#     raise ValueError('key already exists')
# else:
#     score_board[score_type] = np.mean(cv_score)

# lgb = LGBMRegressor(device='GPU',**params).fit(X_scaled, y)


# feat importance plot
feat_importance['avg'] = feat_importance.mean(axis=1)
feat_importance = feat_importance.sort_values(by='avg',ascending=True)
pal=sns.color_palette("plasma_r", 40).as_hex()[2:]


fig=go.Figure()
for i in range(len(feat_importance.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance['avg'][i], 
                       line_color=pal[::-1][i],opacity=0.7,line_width=4))
fig.add_trace(go.Scatter(x=feat_importance['avg'], y=feat_importance.index, mode='markers', 
                         marker_color=pal[::-1], marker_size=8,
                         hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
fig.show()

 




In [ ]:
models

In [ ]:
ensemble = EnsembleModel(models)
ensemble.predict()

In [ ]:
# base model - lgbm - only 130 days for training
# test - time-series cv 5-fold ensemble
class EnsembleModel:
    def __init__(self, models: list):
        self.models = models
    
    def predict(self, x):
        predicted = np.zeros((len(x), len(self.models)))
        for i, model in enumerate(self.models):
            predicted[:, i] = model.predict(x)
        return np.sum(predicted, axis=1) / len(self.models)
        
def group_timeseriesCV(grid, train_size=200, test_size=60):
    # 6-fold
    cv_size = train_size + test_size
    for i in range(4):
        cv_train = grid[cv_size*i:cv_size*(i+1)][:train_size]
        cv_test = grid[cv_size*i:cv_size*(i+1)][-test_size:]
        yield cv_train, cv_test


grid = train['Date'].unique()[90:] #90일의 false data 고려하지 않기
tscv = TimeSeriesSplit(n_splits=10, test_size=60)

k = 0
feat_importance = pd.DataFrame()

cv_score = []
# for train_idx, eval_idx in group_timeseriesCV(grid): #  #

for train_idx, eval_idx in tscv.split(grid):
    train_idx = grid[train_idx]
    eval_idx = grid[eval_idx]

    k += 1
    t0 = time.time()
    print(f'========Training fold {k}========')
   
    print('training size:', len(train_idx), '  test size:', len(eval_idx))
    
    print("Train Date range: {} to {}".format(train_idx.min(),train_idx.max()))
    print("Valid Date range: {} to {}".format(eval_idx.min(),eval_idx.max()))
    

    X, y = X_scaled.copy(), y.copy()
    # X, y = X_lgb.copy(), y.copy()

    X_train, X_test = X.loc[train_idx].reset_index(drop=True), X.loc[eval_idx].reset_index(drop=True)
    y_train, y_test = y.loc[train_idx].reset_index(drop=True), y.loc[eval_idx].reset_index(drop=True)

    
    # lgb = LGBMRegressor(device='GPU')
    # lgb.fit(
    #     X_train, 
    #     y_train,
    #     eval_set = [(X_test, y_test)],
    #     callbacks=[
    #             early_stopping(stopping_rounds=20)
    #         ]
    #     )
    
    

    # feat_importance["Importance_Fold"+str(k)]=lgb.feature_importances_
    # feat_importance.set_index(X_train.columns, inplace=True)

    y_pred = ensemble.predict(X_test)
   
    
    # Sharpe Ratio 
    df_sharpe = pd.DataFrame([])
    df_sharpe['Date'] = X.loc[eval_idx].index
    df_sharpe['predict'] = y_pred
    df_sharpe['Target'] = y_test
    df_sharpe['Rank'] = (df_sharpe.groupby('Date')['predict'].rank(method='first', ascending=False)-1).astype(int)
#         display(X_test.groupby('Date')['Rank'].min())
#         display(X_test.groupby('Date')['Rank'].max())
#         display(X_test)
    eval_score = calc_spread_return_sharpe(df_sharpe)
    cv_score.append(eval_score)

    print(f'Fold {k} evaluated in {time.time() - t0 :.3f}s')
    print(f'Fold {k} sharpe ratio score: {eval_score :.6f}')
    
print(f'cv mean sharpe ratio score: {np.mean(cv_score) :.6f}')

# score_type = 'scaled_6fold_basicfeatonly'
# if score_type in score_board.keys():
#     raise ValueError('key already exists')
# else:
#     score_board[score_type] = np.mean(cv_score)

# lgb = LGBMRegressor(device='GPU',**params).fit(X_scaled, y)


# feat importance plot
# feat_importance['avg'] = feat_importance.mean(axis=1)
# feat_importance = feat_importance.sort_values(by='avg',ascending=True)
# pal=sns.color_palette("plasma_r", 40).as_hex()[2:]


# fig=go.Figure()
# for i in range(len(feat_importance.index)):
#     fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance['avg'][i], 
#                        line_color=pal[::-1][i],opacity=0.7,line_width=4))
# fig.add_trace(go.Scatter(x=feat_importance['avg'], y=feat_importance.index, mode='markers', 
#                          marker_color=pal[::-1], marker_size=8,
#                          hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
# fig.show()

 




In [ ]:
feat_importance

In [ ]:
feat_importance

In [ ]:
feat_importance

In [ ]:
feat_importance.index

In [ ]:
score_board

In [ ]:
# del data, train, supp_data, y, X_scaled
# gc.collect()

In [ ]:
# local-api
sys.path.insert(0, '../input/jpx-local-api')
from local_api import local_api

myapi = local_api('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files', 
                 start_date='2021-11-29', end_date='2022-02-25')
env = myapi.make_env()
iter_test = env.iter_test()
for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(tqdm(iter_test)):
#     t0 = time.time()
    
    # 이전 데이터와 합치고 최근 140일치만 이용한다
    today = prices.iloc[0]['Date']
    lastday = str(pd.to_datetime(today) - pd.DateOffset(140))
    
    if i == 0:
        close_df = pd.concat([
            train_with_supp.loc[
                (train_with_supp['Date'] > lastday) & (train_with_supp['Date'] < today), 
                    ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']],
                prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']]
            ]).reset_index(drop=True)
        
    else:
        close_df = pd.concat([
            close_df.loc[
                (close_df['Date'] > lastday) & (close_df['Date'] < today), 
                    ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']],
                prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']]
            ]).reset_index(drop=True)
        
    # 최근 130개 데이터만 가지고 train 한다. 
    feat = add_features_infer(prices, close_df)
    X = preprocess_inference(feat, trained_scalers)
    
    # display(X)
    # X, y
    X['predict'] = lgb.predict(X)
    X['Rank'] = (X['predict'].rank(method='average', ascending=False)-1).astype(int)
    sample_prediction['Rank'] = X['Rank'].values
    
    # check Rank
    assert sample_prediction["Rank"].notna().all()
    assert sample_prediction["Rank"].min() == 0
    assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
    
#     display(sample_prediction)
    env.predict(sample_prediction)
#     print(time.time() - t0)

In [ ]:
env.score()

In [ ]:

# test_data = train_with_supp.copy()

# test_sec_list = divideSecurities(test_data)
# test_df_added = add_features_train(test_sec_list)

# test_X = preprocess_inference(test_df_added, trained_scalers)
# test_X = test_X[(test_X.index <= '2022-02-25') & (test_X.index >= '2021-11-29')]
# test_y = test_df_added.loc[(test_df_added['Date'] <= '2022-02-25') &
#                            (test_df_added['Date'] >= '2021-11-29'), 'Target']


# test_X['predict'] = lgb.predict(test_X)
# test_X['Target'] = test_y.values
# test_X['Rank'] = (test_X.groupby('Date')['predict'].rank(method='first', ascending=False)-1).astype(int)
# calc_spread_return_sharpe(test_X)

pd.to_datetime 최소한도로 쓸것. 연산 cost가 너무 큼. 특히 inference 단계에서는 쓰지 않기.

In [ ]:
# import jpx_tokyo_market_prediction
# env = jpx_tokyo_market_prediction.make_env()
# iter_test = env.iter_test()

In [ ]:
# for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(iter_test):
# #     t0 = time.time()
    
#     # 이전 데이터와 합치고 최근 140일치만 이용한다
#     today = prices.iloc[0]['Date']
#     lastday = str(pd.to_datetime(today) - pd.DateOffset(140))
    
#     if i == 0:
#         close_df = pd.concat([
#             train_with_supp.loc[
#                 (train_with_supp['Date'] > lastday) & (train_with_supp['Date'] < today), 
#                     ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']],
#                 prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']]
#             ]).reset_index(drop=True)
        
#     else:
#         close_df = pd.concat([
#             close_df.loc[
#                 (close_df['Date'] > lastday) & (close_df['Date'] < today), 
#                     ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']],
#                 prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume']]
#             ]).reset_index(drop=True)
        

#     feat = add_features_infer(prices, close_df)
#     X = preprocess_inference(feat, trained_scalers)

#     # X, y
#     X['predict'] = lgb.predict(X)
#     X['Rank'] = (X['predict'].rank(method='first', ascending=False)-1).astype(int)
#     sample_prediction['Rank'] = X['Rank'].values
    
#     # check Rank
#     assert sample_prediction["Rank"].notna().all()
#     assert sample_prediction["Rank"].min() == 0
#     assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
    
# #     display(sample_prediction)
#     env.predict(sample_prediction)
# #     print(time.time() - t0)